1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면 (인풋이 길면) 답변 생성이 오래거림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200, # 문서를 겹치게 하면 유사도 검색을 할 때 우리가 원하는 문서를 가지고 올 확률을 높임
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter)


/Users/heewongwak/.pyenv/versions/inflearn-llm-application/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # OpenAIEmbeddings의 로드를 위해서 필요

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
%pip install langchain-chroma

You should consider upgrading via the '/Users/heewongwak/.pyenv/versions/inflearn-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_chroma import Chroma

database = Chroma.from_documents(
  documents=document_list,
  embedding=embeddings,
  collection_name="tax_collection",
  persist_directory="./chroma_db"
)
# database = Chroma(
#   collection_name="tax_collection",
#   persist_directory="./chroma_db",
#   embedding_function=embeddings
# )


In [6]:
query = "연봉 1억인 직장인의 소득세는 얼마인가요?" # 존댓말로 하면 답변이 더 좋을 수 있다? 라는 썰이 있다고 한다.
# retrieved_docs = database.similarity_search(query, k=3) # 내 쿼리에 맞춰 유사도 검색을 하는데 관련 문서(데이터)를 3개를 가져온다.

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [17]:
# 페르소나를 주면 좋다?
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

[Question]
{query}
"""

ai_message = llm.invoke(prompt)

ai_message.content


'연봉 1억 원인 직장인의 소득세를 계산하기 위해서는 소득세율과 공제 항목을 고려해야 합니다. 한국의 소득세는 누진세율 구조를 가지고 있으며, 소득 구간에 따라 세율이 다르게 적용됩니다. 2023년 기준으로 소득세율은 다음과 같습니다:\n\n- 1,200만 원 이하: 6%\n- 1,200만 원 초과 ~ 4,600만 원 이하: 15%\n- 4,600만 원 초과 ~ 8,800만 원 이하: 24%\n- 8,800만 원 초과 ~ 1억 5천만 원 이하: 35%\n- 1억 5천만 원 초과 ~ 3억 원 이하: 38%\n- 3억 원 초과 ~ 5억 원 이하: 40%\n- 5억 원 초과: 42%\n\n연봉 1억 원에 대한 소득세를 계산하면 다음과 같습니다:\n\n1. 1,200만 원까지: 1,200만 원 * 6% = 72만 원\n2. 1,200만 원 초과 ~ 4,600만 원까지: (4,600만 원 - 1,200만 원) * 15% = 510만 원\n3. 4,600만 원 초과 ~ 8,800만 원까지: (8,800만 원 - 4,600만 원) * 24% = 1,008만 원\n4. 8,800만 원 초과 ~ 1억 원까지: (1억 원 - 8,800만 원) * 35% = 420만 원\n\n총 소득세 = 72만 원 + 510만 원 + 1,008만 원 + 420만 원 = 2,010만 원\n\n이 계산은 기본적인 소득세 계산이며, 실제로는 각종 공제 항목(예: 인적 공제, 특별 공제 등)을 적용하여 최종 소득세가 결정됩니다. 따라서, 정확한 소득세 금액을 알기 위해서는 개인의 구체적인 상황에 따른 공제 항목을 고려해야 합니다.'

In [8]:
%pip install -U langchain langchainhub --quiet

You should consider upgrading via the '/Users/heewongwak/.pyenv/versions/inflearn-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/Users/heewongwak/.pyenv/versions/inflearn-llm-application/lib/python3.9/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  retriever=database.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

In [11]:
ai_message = qa_chain({"query": query})
ai_message

/var/folders/p6/m5wrxk0n7b7gtlgp_tv_qj_00000gn/T/ipykernel_95369/2125453911.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


{'query': '연봉 1억인 직장인의 소득세는 얼마인가요?',
 'result': '죄송하지만, 제공된 문맥에서는 연봉 1억 원인 직장인의 소득세를 계산할 수 있는 구체적인 세율이나 계산 방법이 포함되어 있지 않습니다. 소득세 계산을 위해서는 소득세율표와 공제 항목 등을 참조해야 합니다. 추가적인 정보가 필요합니다.'}